In [1]:
import sys
sys.path.insert(1, '../../../..')

In [2]:
import optuna
from optuna.trial import TrialState
from optuna.samplers import TPESampler

import torch
import torch.optim as optim
import torch.nn as nn

import numpy as np
import sklearn
import copy

from methods.PINN import PINN
from methods.methodsDataset.PINNDataset import PINNDataset
from methods.DataDrivenMethods import DDMethod
from solvers.Solver import Solver

In [4]:
params_solver = {'equation': 'Poisson', 'domain': [0., 1.], 'D': 1e-3, 'nx': 101}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
params_PINN = {'layer_dims': None, 'activations': None, 'device': device, 'seed': 123}

In [5]:
def delta(y, x, dy = 1., dx = 0.) :
    """    
    y : int, float or ndarray of size 1
    x : ndarray
    
    return dy if x = y and dx otherwise
    """
    if torch.is_tensor(y) or torch.is_tensor(y):
        return torch.where(x == y, dy, dx)
    return np.where(x == y, dy, dx)

In [15]:
params_solver['source_term'] = lambda y, x : delta(y, x)
params_solver['y'] = None

In [16]:
solver = Solver(params={'solver': params_solver})

In [17]:
Y_list = solver.x[20:81].reshape(-1, 1)

In [18]:
U_sols = []
for y in Y_list:
    solver.change_y(new_y= y)
    U_sols.append(solver.solve())
U_sols = np.stack(U_sols)

In [19]:
Y_train, Y_val, U_train, U_val = sklearn.model_selection.train_test_split(Y_list, U_sols, test_size=0.2, random_state=123)

In [21]:
nx = params_solver['nx']

x = torch.Tensor(solver.x).view(-1, 1)
X_train = x.repeat(Y_train.shape[0], 1)
X_val = x.repeat(Y_val.shape[0], 1)


Y_train_repeated = torch.Tensor(Y_train.repeat(nx, 1).reshape(-1, 1))
Y_val_repeated = torch.Tensor(Y_val.repeat(nx, 1).reshape(-1, 1))


YX_train = torch.cat((Y_train_repeated, X_train), dim=1)
YX_val = torch.cat((Y_val_repeated, X_val), dim=1)
YU_train = torch.Tensor(U_train).view(-1, 1)
YU_val = torch.Tensor(U_val).view(-1, 1)

In [23]:
def get_dataset():
    return YX_train, YX_val, YU_train, YU_val

In [24]:
def loss_fn(x, y = 0):
    return torch.square(y - x).mean()

In [25]:
def define_model(trial, input_size, output_size):
    n_layers = trial.suggest_int("n_layers", 1, 5)
    layers = [input_size]
    activations = []
    for i in range(n_layers):
        out_features = trial.suggest_int("units_l{}".format(i), 8, 40)
        layers += [out_features]
        
    layers += [output_size]
    params_PINN_trial = copy.deepcopy(params_PINN)
    params_PINN_trial['layer_dims'] = layers
    params_PINN_trial['activations'] = 'tanh'
    return PINN(params={'solver':params_solver, 'method':params_PINN_trial})

In [11]:
def objective(trial):
    # Generate the model.
    
    model = define_model(trial, 2, 1)
    # Generate the optimizers.
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    
    YX_train, YX_val, YU_train, YU_val = get_dataset()
    
    torch.manual_seed(params_PINN['seed'])
    
    n_epochs = 10000
    hyperparameters = {'lr': lr, 'epochs': n_epochs, 'optimizer': 'Adam'}
    # Training of the model.
    model.fit(hyperparameters = hyperparameters, 
              DX_train = YX_train, DX_val = YX_val, U_val = YU_v, U_train = U_train)
    
    val_loss = model.loss_dict['val']
    trial.report(min(val_loss), n_epochs)

    return min(val_loss)

In [12]:
sampler = TPESampler(seed=params_PINN['seed'])
study = optuna.create_study(direction="minimize", sampler=sampler)

[I 2023-08-09 16:23:59,270] A new study created in memory with name: no-name-0e53be1c-8192-45ec-be8e-6174841e18ae


In [13]:
study.optimize(objective, n_trials=3)

[tr : 5.9e-05, val : 5.3e-06]: 100%|██████| 10000/10000 [02:46<00:00, 60.03it/s]
[I 2023-08-09 16:26:46,294] Trial 0 finished with value: 5.273067017697031e-06 and parameters: {'n_layers': 4, 'units_l0': 17, 'units_l1': 15, 'units_l2': 26, 'units_l3': 31, 'lr': 0.00018591711878786343}. Best is trial 0 with value: 5.273067017697031e-06.
[tr : 4.1e-05, val : 6.9e-06]: 100%|██████| 10000/10000 [03:44<00:00, 44.48it/s]
[I 2023-08-09 16:30:31,131] Trial 1 finished with value: 6.561074769706465e-06 and parameters: {'n_layers': 5, 'units_l0': 30, 'units_l1': 23, 'units_l2': 20, 'units_l3': 19, 'units_l4': 32, 'lr': 0.00020687915518060283}. Best is trial 0 with value: 5.273067017697031e-06.
[tr : 1.3e-03, val : 5.6e-05]: 100%|█████| 10000/10000 [00:49<00:00, 201.53it/s]
[I 2023-08-09 16:31:20,756] Trial 2 finished with value: 5.622710887109861e-05 and parameters: {'n_layers': 1, 'units_l0': 21, 'lr': 0.0016367645761140052}. Best is trial 0 with value: 5.273067017697031e-06.


In [14]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  5.273067017697031e-06
  Params: 
    n_layers: 4
    units_l0: 17
    units_l1: 15
    units_l2: 26
    units_l3: 31
    lr: 0.00018591711878786343


In [15]:
activations = []
layer_dims = []

for key, value in trial.params.items():
    if key.split('_')[0] == 'activation':
        activations.append(value)
    elif key.split('_')[0] == 'units':
        layer_dims.append(value)
    elif key == 'optimizer':
        optimizer = value
    elif key == 'lr':
        lr = value

Use this to fill config_step_1.py

In [16]:
print(activations, '\n', layer_dims, '\n', '\n', lr)

[] 
 [17, 15, 26, 31] 
 
 0.00018591711878786343
